In [1]:
import pandas as pd
#import numpy as np
#from scipy import linalg
import sys
import os
from qkids import Week
from qkids.DatabasePool import get_schedule_connection
sys.path.append("..")
#from schedule.DataFarm import DataFarm
import matplotlib.pyplot as plt
from schedule.Lesson import DataFrameLesson as Lesson

In [29]:
week = Week().get_previous_week()
sql = "select begin_at, lesson_id, schedule_id, room_type_id as roomtype, sum(student_count) student, \
        sum(max_student_count) max_student from schedules s join rooms r on s.id = r.schedule_id and \
        klass_id is null and status = 1 and r.is_internal = 0 where begin_at in {} and room_type_id \
        in (2,5) group by schedule_id".format(week.slots_sqlstring)
df = pd.read_sql(sql, get_schedule_connection())
lesson = Lesson()
df['course'] = df.lesson_id.apply(lesson.get_course_by_lesson)
df

,begin_at,lesson_id,schedule_id,roomtype,student,max_student,course
0,2019-05-04 09:20:00,131,421246,2,3.0,4.0,5
1,2019-05-04 18:40:00,131,421267,5,2.0,2.0,5
2,2019-05-05 19:20:00,131,421270,5,2.0,2.0,5
3,2019-05-04 20:00:00,131,421278,2,4.0,4.0,5
4,2019-05-05 20:40:00,131,421282,2,4.0,4.0,5
5,2019-04-29 19:20:00,76,2534956,2,8.0,8.0,2
6,2019-05-01 19:20:00,78,2534957,2,10.0,12.0,2
7,2019-05-03 19:20:00,79,2534958,2,5.0,8.0,2
8,2019-04-29 19:20:00,178,2582032,2,4.0,4.0,4
9,2019-05-01 19:20:00,179,2582033,2,1.0,4.0,4


### 课程 * 版型  满班率

In [30]:
c = df.groupby(['course', 'roomtype']).sum()
c['rate'] = c.student/c.max_student
d = c.rate.unstack(level=1)
d.columns = ['4人班','2人班']
d

,4人班,2人班
course,,
1,0.924495,0.819661
2,0.916440,0.830031
3,0.877533,0.810658
4,0.834579,0.751656
5,0.736040,0.727027
6,0.702708,0.670270
7,0.632859,0.628440
8,0.631538,0.613402
9,0.551247,0.583333


### 时间段满班率

In [38]:
e = df.groupby(['begin_at', 'course']).sum()

In [41]:
e['rate'] = e.student/e.max_student
e.rate.unstack(level=1)

course,1,2,3,4,5,6,7,8,9,10,11,14,15,43
begin_at,,,,,,,,,,,,,,
2019-04-29 18:40:00,0.972131,0.955078,0.915865,0.826087,0.758475,0.550000,0.569149,0.454545,0.406250,0.367925,0.300000,0.976744,0.870968,0.985000
2019-04-29 19:20:00,0.984300,0.971963,0.931507,0.929752,0.829114,0.655844,0.632812,0.593220,0.500000,0.333333,0.406250,0.981884,0.908537,0.964844
2019-04-29 20:00:00,0.981203,0.983165,0.968447,0.943089,0.852632,0.721591,0.683824,0.564286,0.601852,0.636364,0.350000,0.980989,0.970238,0.981884
2019-04-29 20:40:00,0.971366,0.969048,0.958065,0.934109,0.816667,0.771739,0.686813,0.635135,0.556338,0.500000,0.318182,0.977011,0.898305,0.972727
2019-04-30 18:40:00,0.932343,0.901210,0.916244,0.830065,0.676471,0.608491,0.505882,0.567073,0.391304,0.426829,0.295455,0.948925,0.852564,0.973214
2019-04-30 19:20:00,0.945578,0.962209,0.922170,0.834416,0.722772,0.686869,0.523810,0.625000,0.491525,0.511111,0.442308,0.966912,0.911111,0.977273
2019-04-30 20:00:00,0.975669,0.945338,0.956897,0.916149,0.847222,0.796117,0.687500,0.724359,0.666667,0.560000,0.321429,0.964000,0.885714,0.977528
2019-04-30 20:40:00,0.960630,0.961066,0.968586,0.947853,0.836066,0.794737,0.831461,0.753521,0.618056,0.613208,0.590909,0.952381,0.915493,0.973333
2019-05-01 18:40:00,0.762918,0.821429,0.725877,0.678125,0.531496,0.539604,0.401163,0.500000,0.345455,0.340426,0.328571,0.779412,0.791667,NaN
